In [7]:

import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [10]:
def load_data(folder):
    images = []
    labels = []
    for sub_folder in os.listdir(folder):
        sub_folder_path = os.path.join(folder, sub_folder)
        for image_file in os.listdir(sub_folder_path):
            image_path = os.path.join(sub_folder_path, image_file)
            try:
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
                if image is None:
                    print(f"Warning: Unable to load image '{image_path}'")
                    continue  # Skip this image and move to the next one
                image = cv2.resize(image, (28, 28))  # Resize image to 28x28
                images.append(image)
                labels.append(sub_folder)
            except Exception as e:
                print(f"Error loading image '{image_path}': {str(e)}")
    return np.array(images), np.array(labels)


In [11]:
# Load train and test data
train_images, train_labels = load_data('dataset\data2\testing_data')
test_images, test_labels = load_data('dataset\data2\training_data')

# Preprocess images
train_images = train_images.astype('float32') / 255.0  # Normalize pixel values
test_images = test_images.astype('float32') / 255.0

In [12]:
# Convert labels to one-hot encoding
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

In [13]:
# Split training data into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(36, activation='softmax')  # 36 classes (0-9, A-Z)
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))


Epoch 1/10
516/516 [==============================] - 16s 29ms/step - loss: 1.0605 - accuracy: 0.7138 - val_loss: 0.4362 - val_accuracy: 0.8878
Epoch 2/10
516/516 [==============================] - 18s 35ms/step - loss: 0.3259 - accuracy: 0.9141 - val_loss: 0.2970 - val_accuracy: 0.9186
Epoch 3/10
516/516 [==============================] - 15s 29ms/step - loss: 0.2373 - accuracy: 0.9306 - val_loss: 0.2900 - val_accuracy: 0.9249
Epoch 4/10
516/516 [==============================] - 14s 26ms/step - loss: 0.1840 - accuracy: 0.9433 - val_loss: 0.2344 - val_accuracy: 0.9338
Epoch 5/10
516/516 [==============================] - 14s 26ms/step - loss: 0.1485 - accuracy: 0.9516 - val_loss: 0.2052 - val_accuracy: 0.9389
Epoch 6/10
516/516 [==============================] - 14s 26ms/step - loss: 0.1234 - accuracy: 0.9574 - val_loss: 0.1774 - val_accuracy: 0.9508
Epoch 7/10
516/516 [==============================] - 14s 26ms/step - loss: 0.1084 - accuracy: 0.9622 - val_loss: 0.1866 - val_accuracy:

In [16]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

102/102 [==============================] - 1s 8ms/step - loss: 0.0537 - accuracy: 0.9756
Test accuracy: 0.9756247997283936


In [17]:
import cv2
import numpy as np

# Function to preprocess a single image
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
        if image is None:
            print(f"Warning: Unable to load image '{image_path}'")
            return None
        image = cv2.resize(image, (28, 28))  # Resize image to 28x28
        image = image.astype('float32') / 255.0  # Normalize pixel values
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        return image
    except Exception as e:
        print(f"Error loading or preprocessing image '{image_path}': {str(e)}")
        return None

# Function to predict the class of a single image
def predict_image_class(image_path, model, label_binarizer):
    image = preprocess_image(image_path)
    if image is None:
        return None
    predicted_probabilities = model.predict(image)
    predicted_class_index = np.argmax(predicted_probabilities)
    predicted_class = label_binarizer.classes_[predicted_class_index]
    return predicted_class

# Load trained model and label binarizer
# Assuming `model` and `label_binarizer` are already defined and trained

# Path to the image you want to predict
image_path = '/content/data/training_data/1/11737.png'

# Predict the class of the image
predicted_class = predict_image_class(image_path, model, label_binarizer)

# Print the predicted class
if predicted_class is not None:
    print("Predicted Class:", predicted_class)


1/1 [==============================] - 0s 280ms/step
Predicted Class: 1
